In [5]:

# from scipy.signal import savgol_filter

# # 平滑
# x_smooth = savgol_filter(x, window_length=11, polyorder=3)

# # 一階導數
# dx = savgol_filter(x, 11, 3, deriv=1, delta=1/fs)

# # 二階導數（加速度）
# ddx = savgol_filter(x, 11, 3, deriv=2, delta=1/fs)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


plt.rcParams['figure.dpi'] = 120
plt.rcParams['font.size'] = 11

1. 先用 SG 濾波（保留形狀、避免破壞振動頻率）
2. 用 SG 求速度、加速度
3. 對加速度用 Butterworth 低通（把二階導數放大的高頻 noise 清掉）

給定一excel檔案，其包含高度(t)及位移(x)
這部份的code要重寫!!! 沒有時間

In [ ]:
# 依高度由低到高排列
files = {
500: "H500.csv",
1000: "H1000.csv",
1500: "H1500.csv",
2000: "H2000.csv",
2500: "H2500.csv" # 塔頂
}


# 讀入資料
data = {H: pd.read_csv(f) for H, f in files.items()}


# 確認時間軸（假設一致）
t = data[list(data.keys())[0]]['t'].values

### 找「塔頂最大位移」發生時刻

In [ ]:
H_top = max(files.keys())
delta_top = data[H_top]['x'].values


idx_max = np.argmax(np.abs(delta_top))
t_star = t[idx_max]


print(f"塔頂最大位移發生時間 t* = {t_star:.3f} s")

### 取各高度在 t* 時刻的位移

In [ ]:
H_list = np.array(sorted(files.keys()))
delta_star = []


for H in H_list:
delta_star.append(data[H]['x'].values[idx_max])


delta_star = np.array(delta_star)

### 變形曲線（原始）

In [ ]:
plt.figure()
plt.plot(delta_star, H_list, 'o-', label='Response shape')
plt.axvline(0, color='k', linewidth=0.8)
plt.xlabel("Horizontal displacement δ (cm)")
plt.ylabel("Height H (cm)")
plt.legend()
plt.grid(True)
plt.show()

### 最大反應值 δmax

In [ ]:
delta_max = np.max(np.abs(delta_top))
print(f"塔頂最大位移 δmax = {delta_max:.3f} cm")

### 層間位移 Δδ（彎曲集中位置）

In [ ]:
delta_inter = delta_star[1:] - delta_star[:-1]
H_mid = (H_list[1:] + H_list[:-1]) / 2


plt.figure()
plt.plot(delta_inter, H_mid, 's-')
plt.axvline(0, color='k', linewidth=0.8)
plt.xlabel("Inter-story displacement Δδ (cm)")
plt.ylabel("Height (cm)")
plt.grid(True)
plt.show()

### 正規化位移（Normalized shape）

In [ ]:
delta_norm = delta_star / delta_star[-1]


plt.figure()
plt.plot(delta_norm, H_list, 'o-')
plt.axvline(0, color='k', linewidth=0.8)
plt.xlabel("Normalized displacement δ / δ_top")
plt.ylabel("Height (cm)")
plt.grid(True)
plt.show()

### 主振頻估計（FFT）

In [ ]:
from scipy.fft import rfft, rfftfreq


dt = t[1] - t[0]
N = len(t)


freq = rfftfreq(N, dt)
Y = np.abs(rfft(delta_top))


plt.figure()
plt.plot(freq, Y)
plt.xlim(0, 10) # 視需要調整
plt.xlabel("Frequency (Hz)")
plt.ylabel("Amplitude")
plt.grid(True)
plt.show()